TODO:
1. Match stats correlative to winning
2. Ethan champ correlativee to winning
3. Presence of Marcus
4. Friends' presence to winning

In [ ]:
pip install riotwatcher

     |████████████████████████████████| 57 kB 2.0 MB/s 


In [ ]:
import pandas as pd

In [ ]:
from riotwatcher import LolWatcher
lol_watcher = LolWatcher('')
my_region = 'na1'

In [ ]:
me = lol_watcher.summoner.by_name(my_region, 'LG Hunter 1')

In [ ]:
print(me)

{'id': 'gnzjdGBhD7M6B1kVbnGQyDd0Kd23GhbhURDzS0vj9xOP4Uo', 'accountId': '_WbxvxQmwX7PN75xcF6_kBKL8L7brKeap0Nd1lxAuQvtp5I', 'puuid': '9jU6iIey4R2DWR1M0rFNcGR5Wq_azWl4QwQkXiZI0Cuxn0vHq-C0wfsfXs3VdRyFILawOn1Ul87fqQ', 'name': 'LG Hunter 1', 'profileIconId': 1637, 'revisionDate': 1628577144000, 'summonerLevel': 168}


In [ ]:
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

[{'leagueId': 'd99dc8fa-e3d6-4bf6-8e08-b047d5227918', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GOLD', 'rank': 'II', 'summonerId': 'gnzjdGBhD7M6B1kVbnGQyDd0Kd23GhbhURDzS0vj9xOP4Uo', 'summonerName': 'LG Hunter 1', 'leaguePoints': 75, 'wins': 10, 'losses': 9, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


In [ ]:
matches = lol_watcher.match.matchlist_by_account('na1',me['accountId'],begin_index=0, end_index=100)

In [ ]:
game_id = [] 
game_mode = []
game_duration = []
participant_identities = []
participant_details = []
team_1_stats = []
team_2_stats = []

In [ ]:
for match in matches["matches"]:
  match_details = lol_watcher.match_v4.by_id('na1',match['gameId'])
  game_id.append(match['gameId'])
  game_mode.append(match_details['gameMode']) 
  game_duration.append(match_details['gameDuration']) 
  participant_identities.append(match_details['participantIdentities'])
  participant_details.append(match_details['participants'])
  team_1_stats.append(match_details['teams'][0])
  team_2_stats.append(match_details['teams'][1])
    

In [ ]:
summary_df = pd.DataFrame({
  'game_id':game_id,
  'game_mode':game_mode,
  'game_duration':game_duration,
  'participant_identities':participant_identities,
  'participant_details':participant_details,
  'team_1_stats':team_1_stats,
  'team_2_stats':team_2_stats
})

In [ ]:
team_stats_all = []
player_ids_all = []
my_details_all = []

In [ ]:
for k,row in summary_df.iterrows():

  #Player identity dictionaries
  player_ids = {}
  for i,player in enumerate(row.participant_identities):
    key = 'participantName_'+ (str)(i)
    player_summoner = player['player']['summonerName']
    player_ids[key] = player_summoner
    if player_summoner == 'LG Hunter 1':
      my_gameid = i
  player_ids_all.append(player_ids)

  #My stats
  my_details = None
  for player_details in row.participant_details:
    if(player_details['participantId'] == my_gameid):
      my_details = pd.json_normalize(player_details, sep='_')
      my_team = my_details.teamId[0]
  my_details_all.append(my_details)

  #Team dictionaries
  team_stats = {}
  for key in row.team_1_stats:
    if key not in ['dominionVictoryScore','vilemawKills','bans']:
      if row['team_1_stats']['teamId'] == my_team: #if I'm on team 1
        team_stats[key+'_myTeam'] = row['team_1_stats'][key]
        team_stats[key+'_enemyTeam'] = row['team_2_stats'][key]
      elif row['team_2_stats']['teamId'] == my_team: #if I'm on team 2
        team_stats[key+'_myTeam'] = row['team_2_stats'][key] 
        team_stats[key+'_enemyTeam'] = row['team_1_stats'][key]
  team_stats_all.append(team_stats)


In [ ]:
details_df = pd.concat(my_details_all)
droppable_details = ['participantId','teamId','spell1Id','spell2Id','stats_participantId','stats_win','stats_item0','stats_item1','stats_item2','stats_item3','stats_item4','stats_item5','stats_item6',
                     'stats_doubleKills','stats_tripleKills','stats_quadraKills','stats_pentaKills','stats_unrealKills','stats_magicDamageDealt','stats_physicalDamageDealt','stats_trueDamageDealt',
                     'stats_largestCriticalStrike','stats_totalUnitsHealed','stats_damageSelfMitigated','stats_damageDealtToTurrets','stats_magicalDamageTaken','stats_physicalDamageTaken','stats_trueDamageTaken',
                     'stats_goldSpent','stats_neutralMinionsKilled','stats_neutralMinionsKilledTeamJungle','stats_neutralMinionsKilledEnemyJungle','stats_sightWardsBoughtInGame','stats_wardsPlaced','stats_wardsKilled',
                     'stats_combatPlayerScore','stats_objectivePlayerScore','stats_totalPlayerScore','stats_totalScoreRank','stats_playerScore0','stats_playerScore1','stats_playerScore2','stats_playerScore3',
                     'stats_playerScore4','stats_playerScore5','stats_playerScore6','stats_playerScore7','stats_playerScore8','stats_playerScore9','stats_perk0','stats_perk0Var1','stats_perk0Var2',
                     'stats_perk0Var2','stats_perk1','stats_perk1Var1','stats_perk1Var2','stats_perk1Var3','stats_perk2','stats_perk2Var1','stats_perk2Var2','stats_perk2Var3','stats_perk3','stats_perkPrimaryStyle',
                     'stats_perkSubStyle', 'stats_statPerk0','stats_statPerk1','stats_statPerk2','timeline_participantId','stats_perk3Var1','stats_perk3Var2','stats_perk3Var3','stats_perk4','stats_perk4Var1',
                     'stats_perk4Var2','stats_perk4Var3','stats_perk5','stats_perk5Var1','stats_perk5Var2','stats_perk5Var3','stats_magicDamageDealtToChampions','stats_physicalDamageDealtToChampions','stats_trueDamageDealtToChampions',
                     'stats_turretKills','stats_inhibitorKills','stats_perk0Var3']

details_df = details_df.drop(columns=droppable_details).reset_index()

In [ ]:
player_ids_df = pd.DataFrame(player_ids_all)
team_stats_df = pd.DataFrame(team_stats_all)

In [ ]:
team_stats_df.drop(columns=['teamId_myTeam','teamId_enemyTeam','win_enemyTeam','firstBlood_enemyTeam','firstTower_enemyTeam','firstInhibitor_enemyTeam','firstBaron_enemyTeam','firstDragon_enemyTeam','firstRiftHerald_enemyTeam'],inplace=True)
trimmed_summary_df = summary_df.drop(columns=['participant_identities','participant_details','team_1_stats','team_2_stats'])

In [ ]:
detailed_frame = trimmed_summary_df.join(team_stats_df).join(player_ids_df).join(details_df)

In [ ]:
detailed_frame

,game_id,game_mode,game_duration,win_myTeam,firstBlood_myTeam,firstTower_myTeam,firstInhibitor_myTeam,firstBaron_myTeam,firstDragon_myTeam,firstRiftHerald_myTeam,towerKills_myTeam,towerKills_enemyTeam,inhibitorKills_myTeam,inhibitorKills_enemyTeam,baronKills_myTeam,baronKills_enemyTeam,dragonKills_myTeam,dragonKills_enemyTeam,riftHeraldKills_myTeam,riftHeraldKills_enemyTeam,participantName_0,participantName_1,participantName_2,participantName_3,participantName_4,participantName_5,participantName_6,participantName_7,participantName_8,participantName_9,index,championId,stats_kills,stats_deaths,stats_assists,stats_largestKillingSpree,stats_largestMultiKill,stats_killingSprees,stats_longestTimeSpentLiving,stats_totalDamageDealt,...,stats_totalMinionsKilled,stats_totalTimeCrowdControlDealt,stats_champLevel,stats_visionWardsBoughtInGame,stats_firstBloodKill,stats_firstBloodAssist,stats_firstTowerKill,stats_firstTowerAssist,stats_firstInhibitorKill,stats_firstInhibitorAssist,timeline_creepsPerMinDeltas_10-20,timeline_creepsPerMinDeltas_0-10,timeline_xpPerMinDeltas_10-20,timeline_xpPerMinDeltas_0-10,timeline_goldPerMinDeltas_10-20,timeline_goldPerMinDeltas_0-10,timeline_csDiffPerMinDeltas_10-20,timeline_csDiffPerMinDeltas_0-10,timeline_xpDiffPerMinDeltas_10-20,timeline_xpDiffPerMinDeltas_0-10,timeline_damageTakenPerMinDeltas_10-20,timeline_damageTakenPerMinDeltas_0-10,timeline_damageTakenDiffPerMinDeltas_10-20,timeline_damageTakenDiffPerMinDeltas_0-10,timeline_role,timeline_lane,timeline_creepsPerMinDeltas_30-end,timeline_creepsPerMinDeltas_20-30,timeline_xpPerMinDeltas_30-end,timeline_xpPerMinDeltas_20-30,timeline_goldPerMinDeltas_30-end,timeline_goldPerMinDeltas_20-30,timeline_damageTakenPerMinDeltas_30-end,timeline_damageTakenPerMinDeltas_20-30,timeline_csDiffPerMinDeltas_20-30,timeline_xpDiffPerMinDeltas_20-30,timeline_damageTakenDiffPerMinDeltas_20-30,timeline_csDiffPerMinDeltas_30-end,timeline_xpDiffPerMinDeltas_30-end,timeline_damageTakenDiffPerMinDeltas_30-end
0,4006345119,CLASSIC,1582,Win,False,True,True,False,False,True,7,0,1,0,0,1,2,2,1,0,KingKwan,Silly Rhino,TeriyakiHonda,Zazamachilala,Dark as Day,FLCLBilly,Vonpuppin,LG Hunter 1,Jstocks,Per Solus03,0,86,15,3,9,7,3,3,825,202739,...,32,277,16,0,False,False,False,False,True,False,1.9,0.0,705.0,377.8,615.0,374.9,0.50,-0.90,278.80,45.20,943.4,757.7,122.10,51.20,NONE,JUNGLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4006196181,CLASSIC,1379,Win,True,True,True,False,True,True,9,1,1,0,0,0,3,0,1,0,Rotaerc,PlagueísTheWise,3xtremekiwi,Introspectıon,Faladran,Vonpuppin,LG Hunter 1,Jstocks,Papa BIess,Lancel0tE,0,201,0,0,11,0,0,0,0,19698,...,29,74,11,3,False,False,False,False,False,True,1.4,1.5,400.8,269.5,308.7,169.8,0.45,-0.20,107.45,-4.15,497.9,164.2,-44.50,-29.20,DUO_SUPPORT,BOTTOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4005271454,CLASSIC,1453,Fail,False,False,False,False,False,False,2,6,0,0,0,0,1,2,0,1,tila04,Outplay Mochine,C4Akbar,Cha Min Oh,gawappo,AbsoluteBot,Suck My Cakes,Vonpuppin,LG Hunter 1,Jstocks,0,75,3,5,2,0,1,0,436,76847,...,125,225,13,0,False,False,False,False,NaN,NaN,5.8,3.4,441.7,337.6,357.8,184.0,NaN,NaN,NaN,NaN,1037.0,625.1,NaN,NaN,SOLO,TOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4005073285,CLASSIC,2268,Win,False,False,True,False,False,False,11,5,3,0,2,1,3,2,1,1,Suck My Cakes,Vonpuppin,LG Hunter 1,Jstocks,PepegaSama,Bungo BALLS,Mervis,Chunky Grandma,Zalconx07,PollosTacos,0,86,9,7,11,3,2,3,576,173119,...,182,93,18,3,False,False,False,False,False,False,6.1,7.3,450.1,477.1,485.8,344.9,NaN,NaN,NaN,NaN,786.9,564.4,NaN,NaN,SOLO,TOP,2.400000,3.5,661.8,411.1,608.6,340.3,1893.600000,870.3,NaN,NaN,NaN,NaN,NaN,NaN
4,4004761837,CLASSIC,1859,Win,False,True,True,True,True,True,11,0,3,0,2,0,3,1,1,0,Cpiody,BeechBod,Versparrow,PianoMansHands,Syroxsis,Suck My Cakes,LG Hunter 1,Jstocks,LazyTitan51,bluesbrother123,0,115,1,2,7,0,1,0,1307,122921,...,146,155,15,2,False,False,False,False,False,True,7.1,5.6,498.1,456.8,361.4,257.8,0.70,-0.5

In [ ]:
detailed_frame.game_duration.max()

3160